In [ ]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error


def generate_idhp_frontdoor_binaryT():
    c = 1
    sigma_m = 2
    a = 10
    b = np.array([0, 1, 2, 3, 4])
    probabilities = np.array([0.5,0.2,0.15,0.1,0.05])
    true_ATE = c*a

    obs = pd.read_csv('IHDP_data_CSVs\ihdp_1.csv')
    col_x = []
    for j in range(1,26):
        col_x.append("x"+str(j))
    
    X = obs[col_x]
    T = obs['t']
    d=X.shape[1] #dimension of inputs
    n=X.shape[0] # sample size
    coeff_b = np.random.choice(b, size= d, p=probabilities, replace=True)
    X = preprocessing.scale(X)
    
    M = np.random.multivariate_normal(T,sigma_m *np.diag(np.ones(n)))
    Y = np.random.multivariate_normal(a * M + X @ coeff_b,sigma_m *np.diag(np.ones(n)))
    col = ['t', 'm', 'y']
    #for j in range(1,26):
    #    col.append("x"+str(j))
    df = pd.DataFrame(np.column_stack((T, M, Y)), columns= col )
    return true_ATE, df
    

In [91]:
from dowhy import CausalModel
def ols_ate_frontdoor_binary_t(df_temp):
        # Create a causal model from the data and given common causes.

        nodes_2 = ['t', 'y', 'm', 'U']
        edges_2 = ['tm', 'my', 'Ut', 'Uy']

        # Generate the GML graph
        graph_2 = 'graph [directed 1\n'

        for node in nodes_2:
            graph_2 += f'\tnode [id "{node}" label "{node}"]\n'

        for edge in edges_2:
            graph_2 += f'\tedge [source "{edge[0]}" target "{edge[1]}"]\n'
        
        graph_2 += ']'
        
        model=CausalModel(
                data = df_temp,
                treatment='t',
                outcome='y',
                graph = graph_2,
                )    
        #model.view_model()   
        identified_estimand = model.identify_effect()
        # Using Propensity Score Weighting
        estimate = model.estimate_effect(identified_estimand,
                method_name='frontdoor.two_stage_regression'
        )
        ate= estimate.value
        return ate

In [ ]:
from econml.dml import CausalForestDML
def cf_dml_ate_frontdoor_binary_t(df_temp):

    ate_t_y = np.mean(df_temp[df_temp.t == 1].m) - np.mean(df_temp[df_temp.t == 0].m)
    est = CausalForestDML( random_state=123)
    est.fit(Y = df_temp.y, T = df_temp.m, X = np.array(df_temp.t).reshape(-1, 1))

    ate_m_yt = est.ate(X = np.array(df_temp.t).reshape(-1, 1))

    return ate_t_y * ate_m_yt

In [112]:
def prop_score_ate_frontdoor_binary_t(df):
    from causal_curve import GPS_Regressor
    gps = GPS_Regressor(treatment_grid_num= len(df[df.t == 0]), lower_grid_constraint= 0.0 , upper_grid_constraint= 1.0)

    gps.fit(T = df['m'], X = df['t'], y = pd.to_numeric(df['y']))
    gps_results = gps.calculate_CDRC(0.95)

    mu_01 = np.interp([np.mean(df[df.t == 0]['m']), np.mean(df[df.t == 1]['m'])],gps_results.Treatment,	gps_results.Causal_Dose_Response)
    return mu_01[1] - mu_01[0]


In [5]:
def gp_rbf_ate_frontdoor(df):
    import matplotlib.pyplot as plt
    from pandas.plotting import scatter_matrix
    import numpy as np
    import matplotlib.pyplot as plt
    from sklearn.gaussian_process import GaussianProcessRegressor
    from sklearn.gaussian_process.kernels import RBF, WhiteKernel
    N = len(df)
    # Initialize the GP with the kernel function
    kernel = 1.0 * RBF(length_scale=100.0, length_scale_bounds=(1e-2, 1e3)) \
    + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
    gp= GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=9,  alpha=0.0)
    # Fit the GP to the data
    gp.fit(df[['m', 't']], df['y'])
    y_pred =  []
    #for i in range(N):
    #    y_pred_i =  gp.predict(np.stack(( np.full((N,), df[t].iloc[i]), df[X].values), axis=-1) )
    #    y_pred.append(np.mean(y_pred_i))
    y_pred_0 =  gp.predict(np.stack(( np.full((N,), np.mean(df[df.t == 0]['m'])), df['t'].values), axis=-1) )
    #y_pred.append(np.mean(y_pred_i))
    y_pred_1 =  gp.predict(np.stack(( np.full((N,), np.mean(df[df.t == 1]['m'])), df['t'].values), axis=-1) )
    #y_pred.append(np.mean(y_pred_i))
    return np.mean(y_pred_1) - np.mean( y_pred_0)

In [3]:
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as plt
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel

def average_std(std_arr):
    return 1 / len(std_arr) * np.sqrt(np.square(std_arr).sum()) 


def ate_mean_firstgp(df_temp):
    return np.mean(df_temp[df_temp.t == 1].m) - np.mean(df_temp[df_temp.t == 0].m)







def gp_linear_ate_frontdoor_binary_t_pymc(df):
    import bambi as bmb
    import matplotlib.pyplot as plt
    import numpy as np
    import pandas as pd
    import pymc as pm
    import seaborn as sns

    iterations=5000

    """
    Calculates the Markov Chain Monte Carlo trace of
    a Generalised Linear Model Bayesian linear regression 
    model on supplied data.
    Parameters
    ----------
    df: `pd.DataFrame`
        DataFrame containing the data
    iterations: `int`
        Number of iterations to carry out MCMC for
    """
    # Create the glm using the Bambi model syntax
    model_XZ = bmb.Model("m ~ t", df)
    model_ZY = bmb.Model("y ~ t + m", df)
    # Fit the model using a NUTS (No-U-Turn Sampler) 
    trace_XZ = model_XZ.fit(
        draws= iterations,
        tune=500,
        discard_tuned_samples=True,
        chains=1, 
        progressbar=True)

    trace_ZY = model_ZY.fit(
        draws= iterations,
        tune=500,
        discard_tuned_samples=True,
        chains=1, 
        progressbar=True)
    product_samples = trace_XZ.posterior["t"] * trace_ZY.posterior["m"] 
    return product_samples.to_numpy().mean()

# Generating 1000 realisations of IHDP frontdoor

In [20]:
#for i in range(1000):
#    true_ATE, df = generate_idhp_frontdoor_binaryT()
#    df.to_csv(f"IHDP_frontdooor_data_CSVs/ihdp_fd_{i+1}.csv", index= False)

In [ ]:

import pandas as pd

gp_ate_list = []
for i in range(1000):
    print(i+1 , "  /1000")
    df = pd.read_csv(f"IHDP_frontdooor_data_CSVs/ihdp_fd_{i+1}.csv")
    #cf_dml_ate_list.append(cf_dml_ate_frontdoor_binary_t(df))
    #ols_ate_list.append(ols_ate_frontdoor_binary_t(df))
    #gp_linear_ate_list.append(gp_linear_ate_frontdoor_binary_t_pymc(df))
    gp_ate_list.append(gp_rbf_ate_frontdoor(df))

# Running the ATE estimation


In [150]:
gp_ate_list = []
ipw_ate_list = []
ols_ate_list = []
tmle_ate_list = []
cf_dml_ate_list = []
gp_linear_ate_list = []


true_ATE = 10

for i in range(1000):
    print(i+1 , "  /1000")
    df = pd.read_csv(f"IHDP_frontdooor_data_CSVs/ihdp_fd_{i+1}.csv")
    #cf_dml_ate_list.append(cf_dml_ate_frontdoor_binary_t(df))
    #ols_ate_list.append(ols_ate_frontdoor_binary_t(df))
    #gp_linear_ate_list.append(gp_linear_ate_frontdoor_binary_t_pymc(df))
    gp_ate_list.append(gp_fd_ate_cont_t_better(df))
    #ipw_ate_list.append(prop_score_ate_frontdoor_binary_t(df))
    

564   /1000
565   /1000
566   /1000
567   /1000
568   /1000
569   /1000
570   /1000
571   /1000
572   /1000
573   /1000
574   /1000
575   /1000
576   /1000
577   /1000
578   /1000
579   /1000
580   /1000
581   /1000
582   /1000
583   /1000
584   /1000
585   /1000
586   /1000
587   /1000
588   /1000
589   /1000
590   /1000
591   /1000
592   /1000
593   /1000
594   /1000
595   /1000
596   /1000
597   /1000
598   /1000
599   /1000
600   /1000
601   /1000
602   /1000
603   /1000
604   /1000
605   /1000
606   /1000
607   /1000
608   /1000
609   /1000
610   /1000
611   /1000
612   /1000
613   /1000
614   /1000
615   /1000
616   /1000
617   /1000
618   /1000
619   /1000
620   /1000
621   /1000
622   /1000
623   /1000
624   /1000
625   /1000
626   /1000
627   /1000
628   /1000
629   /1000
630   /1000
631   /1000
632   /1000
633   /1000
634   /1000
635   /1000
636   /1000
637   /1000
638   /1000
639   /1000
640   /1000
641   /1000
642   /1000
643   /1000
644   /1000
645   /1000
646   /1000
647 

### Result for GP Gaussian kernel

In [151]:
import scipy
sem_gp_ate = scipy.stats.sem(np.array(gp_ate_list) - np.full((len(gp_ate_list),),true_ATE))

mean_error_gp_rbf =np.abs(np.mean(np.array(gp_ate_list) - np.full((len(gp_ate_list),),true_ATE))) 
print(mean_error_gp_rbf, "+-", sem_gp_ate)

1.0067260802026994 +- 0.22968746795055522


### Result for Causal forest DML (n = 1000)

In [11]:
import scipy
sem_cf_dml_ate = scipy.stats.sem(np.array(cf_dml_ate_list) - np.full((len(cf_dml_ate_list),),true_ATE))

mean_error_cf_dml =np.abs(np.mean(np.array(cf_dml_ate_list) - np.full((len(cf_dml_ate_list),),true_ATE))) 
print(mean_error_cf_dml, "+-", sem_cf_dml_ate)

nan +- nan


### results for IPW

In [119]:
import scipy
sem_ipw_ate = scipy.stats.sem(np.array(ipw_ate_list) - np.full((len(ipw_ate_list),),true_ATE))

mean_error_ipw_ate =np.abs(np.mean(np.array(ipw_ate_list) - np.full((len(ipw_ate_list),),true_ATE))) 
print(mean_error_ipw_ate, "+-", sem_ipw_ate)

0.04834934822495677 +- 0.05632549920845758


### results for OLS

In [13]:
import scipy
sem_ols_ate = scipy.stats.sem(np.array(ols_ate_list) - np.full((len(ols_ate_list),),true_ATE))

mean_error_ols_ate =np.abs(np.mean(np.array(ols_ate_list) - np.full((len(ols_ate_list),),true_ATE))) 
print(mean_error_ols_ate, "+-", sem_ols_ate)

nan +- nan


# results for GP linear

In [17]:
gp_linear_ate_list

[11.87147400938981, 8.915616771571155, 8.70354537577355, 9.606437510135459, 8.
147477545791245, 11.399575415815498, 11.290513818134158, 7.504881757028366, 9.
772384617904855, 12.27266526359667, 11.041558749887585, 10.493591777069396, 10
.509447574365996, 10.088075474420918, 12.539334302956425, 7.856414278278424, 9
.258470821004332, 9.329655323259924, 9.575504170753934, 8.555028119714278, 7.9
06637914568073, 13.673393485148635, 10.227040601711261, 8.37710738628497, 7.93
8970867781517, 10.312102094925317, 11.139411485243254, 10.565817179514159, 10.
171819170428286, 9.094525790537679, 9.032018336075975, 9.149209055822404, 10.3
26882999048959, 12.551488832146951, 9.43496822563406, 11.767729086640118, 10.4
19185091036386, 9.257762541600279, 11.28698095380459, 5.4711870463706305, 9.82
3707195156299, 11.090839595078142, 10.913248393687272, 7.410742294359759, 8.71
3505773477117, 8.905537881445992, 10.38069814823353, 12.70347607259952, 12.660
655480569668, 9.257866707448432, 7.572116506232319, 

In [18]:
import scipy
gp_ate_array_without_std = gp_linear_ate_list
sem_gp_linear_ate = scipy.stats.sem(np.array(gp_ate_array_without_std) - np.full((len(gp_ate_array_without_std),),true_ATE))

mean_error_gp_linear_ate =np.abs(np.mean(np.array(gp_ate_array_without_std) - np.full((len(gp_ate_array_without_std),),true_ATE))) 
print(mean_error_gp_linear_ate, "+-", sem_gp_linear_ate)

0.04440333804798529 +- 0.0417852140560224
